# TVM 调试器

```{admonition} TVM 配置
- 在 `config.cmake` 中设置 `USE_PROFILER` 为 `ON`。
- 执行 `make tvm`，这样它就会生成 `libtvm_runtime.so`。
```

先载入一些库：

In [1]:
import logging
from dataclasses import dataclass
from typing import Sequence
from PIL import Image
import numpy as np
import mxnet as mx
from mxnet.gluon.model_zoo.vision import get_model
import tvm
from tvm import relay


def transform_image(image):
    """简单的数据预处理"""
    image = np.array(image) - np.array([123.0, 117.0, 104.0])
    image /= np.array([58.395, 57.12, 57.375])
    image = image.transpose((2, 0, 1))
    image = image[np.newaxis, :]
    return image

配置超参数：

In [2]:
@dataclass
class Argument:
    dshape: Sequence[int] = 1, 3, 224, 224
    data_path: str = "../data/test/cat.png"
    log_path = "../.tmp/tvm.log"
    dump_root: str = "../.tmp/tvmdbg"
    model_name: str = "resnet18_v1"
    tvm_input_name: str = "data"

配置日志：

In [3]:
args = Argument()
logging.basicConfig(filename=args.log_path, filemode="w", level=logging.INFO)

数据预处理：

In [4]:
with Image.open(args.data_path) as im:
    im = im.resize((224, 224))
    data = transform_image(im)

加载模型：

In [5]:
block = get_model(args.model_name, pretrained=True)
shape_dict = {args.tvm_input_name: data.shape}
mod, params = relay.frontend.from_mxnet(block, shape_dict)

定义 `softmax`：

In [6]:
func = mod["main"]
func = relay.Function(
    func.params, relay.nn.softmax(func.body), None, func.type_params, func.attrs
)

模型编译：

In [ ]:
out_shape = (1, 1000)
target = tvm.target.Target("llvm", host="llvm")
ctx = tvm.cpu(0)
dtype = "float32"

with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)

创建运行时：

In [8]:
mode = "debug" # "normal", "debug"
if mode == "normal":
    from tvm.contrib.graph_executor import GraphModule
    exe = lib["default"](ctx)
    mlib_proxy = GraphModule(exe)
elif mode == "debug":
    from tvm.contrib.debugger.debug_executor import GraphModuleDebug
    exe = lib["debug_create"]("default", ctx)
    mlib_proxy = GraphModuleDebug(exe, [ctx], lib.graph_json, dump_root=args.dump_root)
else:
    from tvm.contrib.graph_executor import create
    mlib_proxy = create(lib.graph_json, lib, ctx, dump_root=args.dump_root)

前向推理：

In [ ]:
mlib_proxy.set_input(**params)
mlib_proxy.run(data=tvm.nd.array(data.astype(dtype)))
tvm_out = mlib_proxy.get_output(0).asnumpy()

## 节点剖析

In [10]:
mlib_proxy.debug_get_output?

Signature: mlib_proxy.debug_get_output(node, out=None)
Docstring:
Run graph up to node and get the output to out

Parameters
----------
node : int / str
    The node index or name

out : NDArray
    The output array container
File:      /media/pc/data/4tb/lxw/libs/anaconda3/envs/py38/lib/python3.8/site-packages/tvm/contrib/debugger/debug_executor.py
Type:      method


## 缓存与输出

缓存追踪数据：

In [11]:
mlib_proxy.debug_datum.dump_chrome_trace()

缓存全部节点的输出：

In [12]:
mlib_proxy.debug_datum.dump_output_tensor()

加载缓存的节点输出：

In [13]:
params_path = f"{mlib_proxy._dump_path}/output_tensors.params"
with open(params_path, "rb") as fp:
    loaded_params = bytearray(fp.read())
mlib_proxy.load_params(loaded_params)

以 `numpy` 格式获得全部节点的输出张量：

In [14]:
output_tensors = mlib_proxy.debug_datum.get_output_tensors()

打印调试结果：

In [15]:
mlib_proxy.debug_datum.display_debug_result()

Node Name                                                       Ops                                                             Time(us)  Time(%)  Shape                 Inputs  Outputs  Measurements(us)  
---------                                                       ---                                                             --------  -------  -----                 ------  -------  ----------------  
tvmgen_default_fused_nn_contrib_conv2d_NCHWc_add_nn_relu_5      tvmgen_default_fused_nn_contrib_conv2d_NCHWc_add_nn_relu_5      1021.114  7.966    (1, 16, 14, 14, 16)   3       1        [1021.114]        
tvmgen_default_fused_nn_contrib_conv2d_NCHWc_add_1              tvmgen_default_fused_nn_contrib_conv2d_NCHWc_add_1              978.964   7.637    (1, 16, 14, 14, 16)   3       1        [978.964]         
tvmgen_default_fused_nn_contrib_conv2d_NCHWc_add                tvmgen_default_fused_nn_contrib_conv2d_NCHWc_add                944.494   7.368    (1, 4, 28, 28, 32)    3       1  

移除缓存信息：

In [16]:
mlib_proxy.exit()

## 度量单个节点

获取节点数：

In [17]:
len(mlib_proxy.debug_datum.get_graph_nodes())

86

```{rubric} 对序列化计算图中的单个节点进行基准测试
```

这并不进行任何数据传输，而是使用设备上已经存在的数组。

参数：

- `index`：对应于 `debug_datum.get_graph_nodes` 的节点索引。
- `number`: 运行此函数取平均值的次数。这些运行称为度量的单次 `repeat`。
- `repeat` （可选）：重复度量的次数。总的来说，函数将被调用 (1 + number x repeat) 次，其中第一个函数将被预热并将被丢弃。返回的结果包含 `repeat` 开销，每个开销都是 `number` 开销的平均值。
- `min_repeat_ms` （可选）：单次 `repeat` 的最小持续时间，以毫秒为单位。默认情况下，单次 `repeat` 包含 `number` 运行。设置该参数后，参数 `number` 会动态调整，以满足单次 `repeat` 的最小持续时间要求。即，当单次 `repeat` 的运行时间低于此时间时，`number` 参数将自动增加。
- `cooldown_interval_ms` （可选）：由 `repeats_to_cooldown` 定义的重复次数之间的冷却间隔，以毫秒为单位。
- `repeats_to_cooldown`：激活 cooldown 前的重复次数。

运行没有关联函数的节点应该立即返回，并且 runtime 为 `0`：

In [18]:
mlib_proxy.run_individual_node(0, number=1).mean == 0

True

与此同时，实际的函数需要一些时间，如果您运行它多次，则需要更多的时间：

In [19]:
repeat_1_result = mlib_proxy.run_individual_node(1, repeat=1)
assert repeat_1_result.mean > 0

运行多次(10 次)需要的时间应该超过运行 1 次的时间：

In [20]:
repeat_3_results = mlib_proxy.run_individual_node(1, repeat=3)
assert sum(repeat_3_results.results) > sum(repeat_1_result.results)

增加重复次数应该会得到所需结果的数量：

In [21]:
assert len(mlib_proxy.run_individual_node(1, repeat=10).results) == 10

执行 `repeat_ms` 应该使运行时间大于所要求的量 `start = time.time()`：

In [22]:
import time
start = time.time()
mlib_proxy.run_individual_node(1, min_repeat_ms=500)
end = time.time()
elapsed_time_in_seconds = end - start
assert elapsed_time_in_seconds >= 0.5

执行 `cooldown_interval_ms` 会增加执行时间：

In [23]:
start = time.time()
mlib_proxy.run_individual_node(1, repeat=2, min_repeat_ms=500, cooldown_interval_ms=1000)
end = time.time()
elapsed_time_in_seconds_with_def_rep = end - start
assert elapsed_time_in_seconds_with_def_rep >= 3

如果 `repeats_to_cooldown` 不等于 1，则不会在每次重复后触发冷却时间：

In [24]:
start = time.time()
mlib_proxy.run_individual_node(
    1, repeat=2, min_repeat_ms=500, cooldown_interval_ms=1000, repeats_to_cooldown=2
)
end = time.time()
elapsed_time_in_seconds_with_rep_2 = end - start
assert elapsed_time_in_seconds_with_rep_2 >= 2 and (
    elapsed_time_in_seconds_with_rep_2 < elapsed_time_in_seconds_with_def_rep
)